---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [ ]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [ ]:
count = 1
graph_dicts = {}
for graph in P1_Graphs:
    degree_unique_vals = sorted(set(graph.degree().values()), reverse=True)
    graph_dicts['graph-'+str(count)+'-degree-distribution'] = [(i, list(graph.degree().values()).count(i)/float(graph.number_of_nodes())) for i in degree_unique_vals]
    graph_dicts['graph-'+str(count)+'-clustering'] = nx.average_clustering(graph)
    graph_dicts['graph-'+str(count)+'-node_count'] = graph.number_of_nodes()
    graph_dicts['graph-'+str(count)+'-shortest-path'] = nx.average_shortest_path_length(graph)
    count=count+1

In [ ]:
# degree distribution analysis
graph_dicts['graph-1-degree-distribution']

In [ ]:
# # clustering distribution analysis
for i in range(1,6):
    print(graph_dicts['graph-{}-clustering'.format(i)])

In [ ]:
# # shortest path distribution analysis
for i in range(1,6):
    print(graph_dicts['graph-{}-shortest-path'.format(i)])

In [ ]:
# # node count in each graph
for i in range(1,6):
    print(graph_dicts['graph-{}-node_count'.format(i)])

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [ ]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_H','SW_L','PA','SW_H']

In [ ]:
graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [ ]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
# nodes = G.nodes(data=True)
# df = pd.DataFrame(index = G.nodes())
# df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
# df['management-salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
# df['degrees'] = pd.Series(nx.degree(G))
# df['clustering'] = pd.Series(nx.clustering(G))
# # node_attributes_from_graph = [(node[0], node[1]['Department'], node[1]['ManagementSalary']) for node in nodes]

In [ ]:
def mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 1:
            mgmt_emails=mgmt_emails+1
    row['mgmt_communication'] = mgmt_emails
    return row



In [ ]:
def non_mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    non_mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 0:
            non_mgmt_emails=non_mgmt_emails+1
    row['non_mgmt_communication'] = non_mgmt_emails
    return row

In [ ]:
def unknown_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    unknown_emails = 0
    for edge in edges:
        if np.isnan(salary_type_dict[edge[1]]):
            unknown_emails=unknown_emails+1
    row['unknown_communication'] = unknown_emails
    return row

In [ ]:
# df = (df.reset_index()
#         .rename(columns={'index': 'nodes'})
#         .apply(mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
#         .apply(non_mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
# #         .apply(unknown_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
#      )

In [ ]:
# df_training = df[df['management-salary'].isnull() == False]
# df_training.info()

In [ ]:
# df_prediction = df[df['management-salary'].isnull()]
# df_prediction.info()

In [ ]:
# logistic_model.score(X_test,y_test)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df_training.loc[:, df_training.columns !='management-salary'],df_training.loc[:, 'management-salary'], 
#                                                     train_size= 0.8, random_state = 0)

In [ ]:
# from sklearn.linear_model import LogisticRegression
# logistic_model = LogisticRegression(C=1000)
# logistic_model.fit(X_train,y_train)
# logistic_model.score(X_train,y_train)

# prediction = logistic_model.predict(df_prediction.loc[:, df_training.columns !='management-salary'])
# prediction_probas = logistic_model.predict_proba(df_prediction.loc[:, df_training.columns !='management-salary'])
# proba_mgmt_salary = [ proba[1] for proba in prediction_probas]
# mgmt_salary_proba_series = pd.Series(proba_mgmt_salary, index = df_prediction.index)

In [ ]:
def salary_predictions():
    nodes = G.nodes(data=True)
    df = pd.DataFrame(index = G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['management-salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['degrees'] = pd.Series(nx.degree(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    # node_attributes_from_graph = [(node[0], node[1]['Department'], node[1]['ManagementSalary']) for node in nodes]

    df = (df.reset_index()
            .rename(columns={'index': 'nodes'})
            .apply(mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
            .apply(non_mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
    #         .apply(unknown_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
         )
    
    df_training = df[df['management-salary'].isnull() == False]
    df_prediction = df[df['management-salary'].isnull()]
    
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(df_training.loc[:, df_training.columns !='management-salary'],df_training.loc[:, 'management-salary'], 
                                                    train_size= 0.8, random_state = 0)
    
    from sklearn.linear_model import LogisticRegression
    logistic_model = LogisticRegression(C=1000)
    logistic_model.fit(X_train,y_train)
    logistic_model.score(X_train,y_train)
    
    prediction = logistic_model.predict(df_prediction.loc[:, df_training.columns !='management-salary'])
    
    prediction_probas = logistic_model.predict_proba(df_prediction.loc[:, df_training.columns !='management-salary'])
    proba_mgmt_salary = [ proba[1] for proba in prediction_probas]
    mgmt_salary_proba_series = pd.Series(proba_mgmt_salary, index = df_prediction.index)
    # Your Code Here
    
    return mgmt_salary_proba_series

In [ ]:
salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
future_connections.info()

In [ ]:
# comm_net_score = list(nx.cn_soundarajan_hopcroft(G, community='ManagementSalary'))

In [ ]:
# cn_dict = {}
# for cn_score in comm_net_score:
#     cn_dict[(cn_score[0], cn_score[1])] = cn_score[2]

In [ ]:
# jaccard_coeff = list(nx.jaccard_coefficient(G))

In [ ]:
# jacc_dict = {}
# for jacc_score in jaccard_coeff:
#     jacc_dict[(jacc_score[0], jacc_score[1])] = jacc_score[2]

In [ ]:
# future_connections = (future_connections.reset_index()
#                                         .rename(columns={'index' : 'edge'})
#                      )

In [ ]:
# future_connections.head()

In [ ]:
# future_connections['cn-score'] = future_connections['edge'].apply(lambda x : cn_dict[x])

In [ ]:
# future_connections.set_index('edge', drop=True, inplace=True)

In [ ]:
# future_connections.head()

In [ ]:
# from sklearn.model_selection import train_test_split

# future_connections_train = future_connections[future_connections['Future Connection'].isnull() == False]
# future_connections_predict = future_connections[future_connections['Future Connection'].isnull()]

In [ ]:


# X_train, X_test, y_train, y_test = train_test_split(future_connections_train['cn-score'], future_connections_train['Future Connection'], 
#                                                 train_size= 0.8)

# from sklearn.linear_model import LogisticRegression
# logistic_model = LogisticRegression(C=100)
# logistic_model.fit(X_train.reshape(X_train.shape[0],1), y_train.reshape(y_train.shape[0],1))
# logistic_model.score(X_test.reshape(X_test.shape[0],1), y_test.reshape(y_test.shape[0],1))

# # prediction = logistic_model.predict(future_connections_predict.loc[:, df_training.columns !='management-salary'])

# prediction_probas = logistic_model.predict_proba(future_connections_predict['cn-score'].reshape(future_connections_predict['cn-score'].shape[0],1))
# proba_future_connect = [ proba[1] for proba in prediction_probas]
# future_connect_series = pd.Series(proba_mgmt_salary, index = future_connections_predict.index)

In [ ]:
def new_connections_predictions():
    comm_net_score = list(nx.cn_soundarajan_hopcroft(G, community='ManagementSalary'))

    cn_dict = {}
    for cn_score in comm_net_score:
        cn_dict[(cn_score[0], cn_score[1])] = cn_score[2]

    future_connection = (future_connections.reset_index()
                                            .rename(columns={'index' : 'edge'})
                         )

    future_connection['cn-score'] = future_connection['edge'].apply(lambda x : cn_dict[x])

    future_connection.set_index('edge', drop=True, inplace=True)

    from sklearn.model_selection import train_test_split

    future_connections_train = future_connection[future_connection['Future Connection'].isnull() == False]
    future_connections_predict = future_connection[future_connection['Future Connection'].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(future_connections_train['cn-score'], future_connections_train['Future Connection'], 
                                                    train_size= 0.8)

    from sklearn.linear_model import LogisticRegression
    logistic_model = LogisticRegression(C=100)
    logistic_model.fit(X_train.reshape(X_train.shape[0],1), y_train.reshape(y_train.shape[0],1))
    logistic_model.score(X_test.reshape(X_test.shape[0],1), y_test.reshape(y_test.shape[0],1))

    # prediction = logistic_model.predict(future_connections_predict.loc[:, df_training.columns !='management-salary'])

    prediction_probas = logistic_model.predict_proba(future_connections_predict['cn-score'].reshape(future_connections_predict['cn-score'].shape[0],1))
    proba_future_connect = [ proba[1] for proba in prediction_probas]
    future_connect_series = pd.Series(proba_future_connect, index = future_connections_predict.index)
    
    return future_connect_series

In [ ]:
new_connections_predictions()